In [1]:
# process level parameters
import sys

env = 'uat'
if env=='prod':
    mesos_ip = 'mesos://10.35.12.205:5050'
else:
    mesos_ip = 'mesos://10.33.195.18:5050'
    
#2*8+(2*8*10%) = 16+1.6 =17.6 --Memory Calculation
no_of_cpu = 2
max_cores = 2
executr_mem = '16g'
app_name = 'Mahantesh: Baseline_Metrics'

print (no_of_cpu)
print (max_cores)
print (executr_mem)


from pyspark.mllib.stat import Statistics
from pyspark.sql.functions import asc,lit
#warnings.filterwarnings('error')
import pyspark
from datetime import datetime,timedelta
from pyspark.sql import SQLContext
from pyspark.sql.window import Window
from pyspark import SparkContext, SparkConf
conf = pyspark.SparkConf()
#import numpy
import calendar
import time
#import pandas as pd
#import simplejson as json
#import pandas as pd
import numpy as np
import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType, StringType, DecimalType
from pyspark.sql.functions import lit
import simplejson as json
import json, pprint, requests
import re # to use regular expressions
es_nodes = '10.35.12.212'
es_port = '9200'
es_user = 'elastic'
es_pwd = 'bEiilauM3es'
#mesos_ip = 'mesos://10.35.12.5:5050'
#mesos_ip = 'mesos://10.35.12.205:5050'
mesos_ip = mesos_ip
spark.stop()
conf.setMaster(mesos_ip)

conf.set('spark.executor.cores',no_of_cpu)
conf.set('spark.cores.max',max_cores)
#conf.set('spark.memory.fraction','.2')
conf.set('spark.executor.memory',executr_mem)
conf.set('spark.es.scroll.size','10000')
conf.set('spark.network.timeout','600s')
conf.set('spark.sql.crossJoin.enabled', 'true')

conf.set('spark.executor.heartbeatInterval','60s')
conf.set("spark.driver.cores","4")
conf.set("spark.driver.extraJavaOptions","-Xmx4g -Xms4g")

#conf.set("spark.shuffle.blockTransferService", "nio"); 
conf.set("spark.files.overwrite","true");
conf.set("spark.kryoserializer.buffer", "70");
conf.set("spark.driver.extraJavaOptions", "-XX:+UseG1GC");
conf.set("spark.executor.extraJavaOptions", "-XX:+UseG1GC");
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer"); 
conf.set("spark.broadcast.compress", "true");
conf.set("spark.shuffle.compress", "true"); 
conf.set("spark.shuffle.spill.compress", "true");
conf.set("spark.app.name",app_name)
#conf.set("spark.io.compression.codec","org.apache.spark.io.LZ4CompressionCodec");
#conf.set("spark.sql.inMemoryColumnarStorage.compressed", "true"); 
from pyspark.sql.functions import broadcast

conf.set('spark.driver.memory','32g')

conf.set('spark.sql.shuffle.partitions','300')
#conf.set('spark.sql.crossJoin.enabled', 'true')
# conf.set('es.nodes',es_nodes)
# conf.set('es.port',es_port)
conf.set('es.nodes.wan.only','true')
conf.set("spark.sql.autoBroadcastJoinThreshold",-1)

#conf.set('spark.es.net.http.auth.user','Spark')
#conf.set('spark.es.net.http.auth.pass','Jarkpet1Sap3')
#conf.set('spark.num.executors','5')

# conf.set('spark.es.net.http.auth.user', es_user)
# conf.set('spark.es.net.http.auth.pass', es_pwd)

conf.set('spark.es.mapping.date.rich','false')
spark = pyspark.SparkContext(conf=conf)
sqlContext = SQLContext(spark)

# Load Data into PySpark DataFrames
# Prodcom Data Frame
import json, pprint, requests
import pyspark.sql.functions as sf

##Define Required Functions


def documentcount(index_name):
    headers = {'Content-Type': 'application/json'}
    query = {'query':{'match_all':{}}}
    URL = 'http://'+es_user+':'+es_pwd+'@'+es_nodes_tmp+':'+es_port+'/'+index_name+'/_count'
    r = requests.get(URL, headers=headers)
    document_count =r.json().get('count')
    if document_count == 0:
        document_count = 1
    return document_count

def createindex(index_name,number_of_shards,number_of_replicas):
####   Create a Index     #######
    headers = {'Content-Type': 'application/json'}
    index_config= {"settings": { "index.number_of_shards": number_of_shards , "index.number_of_replicas":number_of_replicas}}
    URL = 'http://'+es_user+':'+es_pwd+'@'+es_nodes+':'+es_port+'/'+index_name
    r = requests.put(URL, data=json.dumps(index_config), headers=headers)
    return r


def createmapping(index_name,doc_type,mappings):
### Create a doc-type Mappings    
    URL = 'http://'+es_user+':'+es_pwd+'@'+es_nodes+':'+es_port+'/'+index_name+'/_mappings/'+doc_type+'/'
    headers = {'Content-Type': 'application/json'}
    r = requests.put(URL, data=json.dumps(mappings), headers=headers)
    return r

def deleteindex(index_name):
###delete index if already exists########################################################
    headers = {'Content-Type': 'application/json'}
    query = {"query":{"match_all":{}}}
    URL = 'http://'+es_user+':'+es_pwd+'@'+es_nodes+':'+es_port+'/'+index_name
    r = requests.delete(URL, data=json.dumps(query), headers=headers)
    return r

def last_day_of_month(any_day):
    next_month = any_day.replace(day=28) + datetime.timedelta(days=4)
    return next_month - datetime.timedelta(days=next_month.day)

def last_day_of_previous_month(any_day):
    previous_month = any_day.replace(day = 1)
    return previous_month - datetime.timedelta(days=1)

def first_day_of_last_month(any_day):
    previous_month = any_day.replace(day = 1)
    previous_month = previous_month - datetime.timedelta(days=1)
    return previous_month.replace(day = 1)

def first_day_of_month(any_day):
    if any_day != None:
        return any_day.replace(day = 1)
    else:
        return any_day

def string_to_date(any_day):
    if any_day != None:
        return datetime.datetime.strptime(any_day,'%Y-%m-%d')
    else:
        return any_day
    
# added by SUMIT for dd/mm/yyyy format    
def string_to_date_1(any_day):
    if any_day != None:
        return datetime.datetime.strptime(any_day,'%d/%m/%Y')
    else:
        return any_day    
    

from pyspark.sql.types import IntegerType
def day_of_date(any_date):
    if any_date != None:
        return any_date.day
    else:
        return any_date

def year_of_date(any_date):
    if any_date != None:
        return any_date.year
    else:
        return any_date

from pyspark.sql.types import StringType
def first_ten(any_string):
    if any_string != None:
        return any_string[0:10]
    else:
        return any_string      

def after_ten(any_string):
    if any_string != None:
        return any_string[10:]
    else:
        return any_string 

def first_two(any_string):
    if any_string != None:
        if(len(any_string) == 7):
            return any_string[0:2]
        else:
            return '0'
    else:
        return any_string

def three_four(any_string):
    if any_string != None:
        if(len(any_string) == 7):
            return any_string[2:4]
        else:
            return '0'
    else:
        return any_string

def after_four(any_string):
    if any_string != None:
        if(len(any_string) == 7):
            return any_string[4:7]
        else:
            return '0'
    else:
        return any_string

def lengthSt(any_string):
    if any_string != None:
        return len(any_string)
    else:
        return 0
    
def upperAllColName(df):
    for col in df.columns:
        df=df.withColumnRenamed(col, (col).upper())
    return df    

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
62,None,pyspark,idle,,,✔


SparkSession available as 'spark'.
2
2
16g

In [2]:
url_uat = "jdbc:postgresql://10.33.195.103:5432/gpadmin"
url_prod = "jdbc:postgresql://10.35.12.194:5432/gpadmin"
user_prod="gpspark"
pwd_prod="spark@456"
dbschema_name = "public"
dbschema_name_2="staging"

In [24]:
gscPythonOptions = {
         "url": "jdbc:postgresql://10.35.12.194:5432/gpadmin","user": "gpspark","password": "spark@456",
         "dbschema": "public","dbtable": "customer_gc_genmst_customer",
         "partitionColumn":"row_num","partitions":16,
         "server.port":"1177"}

customer_gc_genmst_customer =sqlContext.read.format("greenplum").options(**gscPythonOptions).load()

In [26]:
#customer_gc_genmst_customer.CreateOrRegisterTempTable("customer_gc_genmst_customer_2")

customer_gc_genmst_customer.createOrReplaceTempView("customer_gc_genmst_customer_2")

In [32]:
customer_gc_genmst_customer_3 = sqlContext.sql("select * \
from customer_gc_genmst_customer_2 where IND_CORP_FLAG='I' and \
date_format(coalesce (DAT_MODIFY_DATE,DAT_INSERT_DATE),'yyyy-mm-dd hh:mm:ss')>='2021-11-01' \
and date_format(coalesce (DAT_MODIFY_DATE,DAT_INSERT_DATE),'yyyy-mm-dd hh:mm:ss')< '2021-12-01'")

In [36]:
column_names = customer_gc_genmst_customer_3.columns

In [39]:
len(column_names)

186

In [40]:
# customer_gc_genmst_customer --> customer_code
# prodcomm -- record_type_desc
email_columns = ['email', 'email_2']
phone_columns = ['mobile_no', 'mobile1', 'mobile2', 'mobile3']

In [44]:
spark.stop()

In [ ]:
customer_gc_genmst_customer_3=customer_gc_genmst_customer_3.toPandas()

import numpy as np
customer_gc_genmst_customer_3['mobile_flag']=np.where((customer_gc_genmst_customer_3['mobile_no'].isna() & customer_gc_genmst_customer_3['mobile1'].isna(),
                                                      customer_gc_genmst_customer_3['mobile2'].isna() & customer_gc_genmst_customer_3['mobile3'].isna()),0,1)

In [ ]:
customer_gc_genmst_customer_4 = customer_gc_genmst_customer_3.withColumn('mobile_flag', \
                                    when((customer_gc_genmst_customer_3.mobile_no.isNull())&\
                                         (customer_gc_genmst_customer_3.mobile1.isNull())&\
                                         (customer_gc_genmst_customer_3.mobile2.isNull())&\
                                         (customer_gc_genmst_customer_3.mobile3.isNull()),0\
                                            .otherwise(1))

In [ ]:
customer_gc_genmst_customer_4 = customer_gc_genmst_customer_3.withColumn('mobile_flag',\
                                                                         F.when(customer_gc_genmst_customer_3.mobile_no.isNull() &\
                                                                                customer_gc_genmst_customer_3.mobile1.isNull() &\
                                                                                customer_gc_genmst_customer_3.mobile2.isNull() &\
                                                                                customer_gc_genmst_customer_3.mobile3.isNull(), 1)\
                                                                         .otherwise(0)

In [ ]:
customer_gc_genmst_customer_5 = customer_gc_genmst_customer_4.withColumn('email_flag', \
                                    when((customer_gc_genmst_customer_3.email.isNull())&\
                                         (customer_gc_genmst_customer_3.email_2.isNull()),0\
                                            .otherwise(1))

In [ ]:
customer_gc_genmst_customer_5.select('customer_code').value_count()